In [1]:
spark.conf.set("spark.sql.shuffle.partitions", 1000)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [2]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import Row
from pyspark.ml.clustering import KMeans, KMeansModel, BisectingKMeans, BisectingKMeansModel
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, PCAModel, ElementwiseProduct
from pyspark.sql.functions import current_date, broadcast
import time

### Standarization and Model load

In [3]:
sparkFeaturePipelineModel=PipelineModel.load("/analytics/bc1_clustering/models/pipeline12DicGrp")

In [4]:
transformer=ElementwiseProduct.load("/analytics/bc1_clustering/models/scaleFinancial12DicGrp")

In [6]:
model=BisectingKMeansModel.load("/analytics/bc1_clustering/models/model2DicGrp")

In [7]:
dfOrden=spark.read.parquet('/analytics/bc1_clustering/tables/dfOrden12DicGrp')

### User - Product dataframe

In [8]:
# HERE WE'D PROCESS THE DATA JUST AS IN THE DATA WRANGLING TO GET BUFFER TICKET USER-PRODUCT 
# AS IT IS SAVED, I'LL JUST LOAD IT FOR THE DEVELOPMENT
dfBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrix11Dic")

### Cluster assignment

In [9]:
dfBufferFeatureTmp=sparkFeaturePipelineModel.transform(dfBuffer)
# dfBufferFeature=sparkFeaturePipelineModel.transform(dfBuffer.withColumnRenamed('','NULO'))

In [10]:
dfBufferFeature=transformer.transform(dfBufferFeatureTmp)

In [11]:
dfSegmentation=model.transform(dfBufferFeature).persist()

In [12]:
dfFinalSegmentation=dfSegmentation.join(broadcast(dfOrden),dfSegmentation.prediction==dfOrden.prediction) \
    .orderBy('orden').select("ACCOUNTPK",dfSegmentation.prediction)

In [13]:
dfFinalSegmentation.persist().show(5)

+---------+----------+
|ACCOUNTPK|prediction|
+---------+----------+
|   380612|        12|
|   530868|        12|
|   564430|        12|
|   632370|        12|
|   833333|        12|
+---------+----------+
only showing top 5 rows



In [14]:
dfFinalSegmentation.count()

4166747

### Save results

In [15]:
dfFinalSegmentation.coalesce(1) \
    .write.csv('/analytics/bc1_clustering/outputs/output12DicGrpx4.csv')

In [ ]:
#time.strftime('%Y%m%d')